In [1]:
#%load_ext autoreload
#%autoreload 2


import netcbs
import numpy as np
import polars as pl

In [2]:
# Print contexts and codebook
print(netcbs.context2types)
print(netcbs.codebook)

{'Family': {301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322}, 'Colleagues': {201}, 'Neighbors': {101, 102}, 'Schoolmates': {501, 502, 503, 504, 505, 506}, 'Housemates': {401, 402}}
{101: 'Neighbor - 10 closest addresses', 102: 'Neighborhood acquaintance - 20 random neighbors within 200 meters', 201: 'Colleague', 301: 'Parent', 302: 'Co-parent', 303: 'Grandparent', 304: 'Child', 305: 'Grandchild', 306: 'Full sibling', 307: 'Half sibling', 308: 'Unknown sibling', 309: 'Full cousin', 310: 'Cousin', 311: 'Aunt/Uncle', 312: 'Partner - married', 313: 'Partner - not married', 314: 'Parent-in-law', 315: 'Child-in-law', 316: 'Sibling-in-law', 317: 'Stepparent', 318: 'Stepchild', 319: 'Stepsibling', 320: 'Married full cousin', 321: 'Married cousin', 322: 'Married aunt/uncle', 401: 'Housemate', 402: 'Housemate - institution', 501: 'Classmate primary education', 502: 'Classmate special education', 503: 'Classmate secondary education', 50

In [3]:
# Create df_sample example
df_sample = pl.DataFrame(
    {
        "RINPERSOON": range(100_000_000, 100_010_000),
        "RINPERSOONS": ["R"]*10_000
    }
)

df_agg = pl.DataFrame(
    {
        "RINPERSOON":   range(100_000_000, 101_000_000),
        "RINPERSOONS":  ["R"]*1_000_000,
        "Income":       30000 + np.random.randn(1_000_000)*5000
    }
)



In [4]:

## How to construct the query
# 1. Start with "Sample ->"
# 2. Then add the relationships between the tables, e.g., "Sample -> Schoolmates[all]".
# In square brackets you can specify the type of the relationships: 
# write [all] for all, or [301,302] for parents and co-parents
# 3. You can add several tables: "Sample -> Schoolmates[all] -> Family[301]"
# 4. Finally, you must specify the column you want to aggregate: "Sample -> Schoolmates[all] -> Family[301] -> Income"

## Other parameters
# df_sample: the sample dataframe (with the people you want to have information on)
# df_agg: the dataframe with the information you want to aggregate. For example, the income of all people in the country
# year: the year of the data you want to use
# agg_func: the aggregation function you want to use. For example, pl.mean or pl.sum
# return_pandas: if True, the function returns a pandas dataframe. If False, it returns a polars dataframe
# lazy: if True, the operations are concatenated lazily and computed at the end. If False, the operations are computed immediately

## Example
query =  "Sample -> Schoolmates[all] -> Family[301,302,303] -> Income"
df = netcbs.transform(query, 
               df_sample = df_sample, 
               df_agg = df_agg, 
               year=2021,
               cbsdata_path='cbsdata/Bevolking', # Path to the CBS data, in this example is synthetic data locally 
               agg_func=pl.mean, 
               return_pandas=False, 
               lazy=True)

df

    

RINPERSOON,RINPERSOONS,Income
i64,str,f64
100000000,"""R""",null
100000001,"""R""",null
100000002,"""R""",34969.390509
100000003,"""R""",null
100000004,"""R""",null
…,…,…
100009995,"""R""",null
100009996,"""R""",null
100009997,"""R""",null
